In [20]:
import urllib.request
import io
import gzip
import json
import pandas as pd
import numpy as np
from itertools import product
from IPython.display import display, HTML
import logging as log

URL='http://data.githubarchive.org/2015-01-01-0.json.gz'
BASE_URL = 'http://data.githubarchive.org/'
URL = 'http://data.githubarchive.org/2017-11-08-1.json.gz'

In [25]:
def read_file(url):
    response = urllib.request.urlopen(url)
    compressed_file = io.BytesIO(response.read())
    decompressed_file = gzip.GzipFile(fileobj=compressed_file)
    
    data = []
    for i, line in enumerate(decompressed_file.readlines()):
        try:
            data.append(json.loads(line))
        except:
            log.warning('Error to parse line {l} from {u}'.format(l=i, u=url))
            
    return data

In [26]:
data = read_file(URL)
len(data)

48192

In [110]:
def filter_data(data):
    VALID_EVENTS = ['PullRequestEvent']
    VALID_ACTIONS = ['opened', 'closed']
    
    filter_events = lambda x: x['type'] in VALID_EVENTS
    filter_actions = lambda x: x['payload']['action'] in VALID_ACTIONS
    filter_langue = lambda x: pd.notnull(x['payload']['pull_request']['base']['repo']['language'])
    
    filtered_data = filter(filter_events, data)
    filtered_data = filter(filter_actions, filtered_data)
    filtered_data = filter(filter_langue, filtered_data)
    
    return filtered_data

In [111]:
filtered = filter_data(data)
len(list(filtered))

294

In [177]:
def get_urls(start_date, end_date):
    days = list(pd.date_range(start_date, end_date, freq='1D'))
    days_str = map(lambda x: x.strftime('%Y-%m-%d'), days)
    days_hours = product(range(0, 1), days_str)
    
    fn_url = lambda x: BASE_URL + x[1] + '-' + str(x[0]) + '.json.gz'
    
    return map(fn_url, days_hours)

In [178]:
urls = get_urls('2017-10-18', '2017-10-18')

In [179]:
x = list(map(read_file, urls))

In [180]:
len(x)

1

In [183]:
df = pd.io.json.json_normalize(x[0])

In [197]:
HTML(pd.DataFrame(df[df['type']=='PullRequestEvent'].iloc[98].T).to_html())

,1940
actor.avatar_url,https://avatars.githubusercontent.com/u/31862127?
actor.display_login,jillianyong
actor.gravatar_id,
actor.id,31862127
actor.login,jillianyong
actor.url,https://api.github.com/users/jillianyong
created_at,2017-10-18T00:02:24Z
id,6730306613
org.avatar_url,NaN
org.gravatar_id,NaN
